In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

# DATA

In [2]:
dmeta = pd.read_csv("../01_data_prep/dataPrep_01_overview_ITS_META.tsv", sep="\t", header=0)
dmeta['SampleID'] = dmeta['long.name'].apply(lambda x: x.replace('.','-').replace("X",""))
print(dmeta.shape)
dmeta.head()

samples = dmeta["SampleID"].drop_duplicates().values
print(len(samples))
#samples

(133, 15)
133


In [3]:
df = pd.read_csv("/mnt/d/NRCan/2022_MicrobiomeQuebecMexico/DATA/ITS/Full/Peuplier_REPRISE_ITS_oct2022.ASV_table_norarefaction_dnNA.guilds.txt", sep='\t', header=0)
df.head()

,#OTU ID,001-ESSI-001-S-Root-ITS,002-ESSI-001-S-Rhizo-ITS,003-ESSI-001-S-Soil-ITS,004-ESSI-002-S-Root-ITS,005-ESSI-002-S-Rhizo-ITS,006-ESSI-002-S-Soil-ITS,007-ESSI-003-S-Root-ITS,008-ESSI-003-S-Rhizo-ITS,009-ESSI-003-S-Soil-ITS,...,taxonomy,Taxon,Taxon Level,Trophic Mode,Guild,Growth Morphology,Trait,Confidence Ranking,Notes,Citation/Source
0,77b72457c1ffe5587d944f4faf895c1f,1360.0,1223.0,941.0,8.0,3291.0,3898.0,662.0,2375.0,1602.0,...,k__Fungi; p__Basidiomycota; c__Agaricomycetes;...,Sebacinaceae,9,Symbiotroph,Ectomycorrhizal,Coralloid,NaN,Probable,Terrestrial forming mycorrhizas with a wide ra...,"b'Cannon PF, Kirk PM. 2007. Fungal Families of..."
1,1b0809805096c16c7a7126f5ae3aef8a,758.0,556.0,550.0,8.0,2972.0,3233.0,394.0,1445.0,1138.0,...,k__Fungi; p__Mortierellomycota; c__Mortierello...,Mortierella,13,Saprotroph-Symbiotroph,Endophyte-Litter Saprotroph-Soil Saprotroph-Un...,Microfungus,NaN,Probable,Saprobic in soil and thus frequenctly isolated...,"b'Cannon PF, Kirk PM. 2007. Fungal Families of..."
2,7b5e91b75899fc74da9b7d623f2b9c6e,681.0,637.0,585.0,4.0,1992.0,2477.0,394.0,1266.0,982.0,...,k__Fungi; p__Basidiomycota; c__Agaricomycetes;...,Tylospora,13,Symbiotroph,Ectomycorrhizal,Corticioid,NaN,Highly Probable,NaN,"b'Rinaldi AC, et al. 2008. Fungal Diversity 33..."
3,a618c5f336416bda99f2f5d289dd74c6,20.0,18.0,0.0,0.0,0.0,41.0,19.0,118.0,164.0,...,k__Fungi; p__Ascomycota; c__Leotiomycetes; o__...,Pseudeurotiaceae,9,Saprotroph,Plant Saprotroph-Wood Saprotroph,Microfungus,NaN,Probable,Saprobic on woody tissue and rotting vegetatio...,"b'Cannon PF, Kirk PM. 2007. Fungal Families of..."
4,ad520cb61c8760c17a08cd2d6327b35f,110.0,184.0,157.0,0.0,625.0,641.0,95.0,441.0,378.0,...,k__Fungi; p__Basidiomycota; c__Tremellomycetes...,-,-,-,-,-,-,-,Unassigned,-


## Abundance and taxonomy tables 

In [4]:
#columns_its = [ele for ele in df.columns if 'ITS' in ele]
columns_its = samples
columns_tax = [ele for ele in df.columns if 'ITS' not in ele]
df_abu = df[columns_its]
df_abu = df_abu.set_index(df['#OTU ID'])
print(df_abu.shape)
df_tax = df[columns_tax]
df_tax = df_tax.set_index('#OTU ID')
print(df_tax.shape)
df_tax.head()

(7320, 133)
(7320, 10)


,taxonomy,Taxon,Taxon Level,Trophic Mode,Guild,Growth Morphology,Trait,Confidence Ranking,Notes,Citation/Source
#OTU ID,,,,,,,,,,
77b72457c1ffe5587d944f4faf895c1f,k__Fungi; p__Basidiomycota; c__Agaricomycetes;...,Sebacinaceae,9,Symbiotroph,Ectomycorrhizal,Coralloid,NaN,Probable,Terrestrial forming mycorrhizas with a wide ra...,"b'Cannon PF, Kirk PM. 2007. Fungal Families of..."
1b0809805096c16c7a7126f5ae3aef8a,k__Fungi; p__Mortierellomycota; c__Mortierello...,Mortierella,13,Saprotroph-Symbiotroph,Endophyte-Litter Saprotroph-Soil Saprotroph-Un...,Microfungus,NaN,Probable,Saprobic in soil and thus frequenctly isolated...,"b'Cannon PF, Kirk PM. 2007. Fungal Families of..."
7b5e91b75899fc74da9b7d623f2b9c6e,k__Fungi; p__Basidiomycota; c__Agaricomycetes;...,Tylospora,13,Symbiotroph,Ectomycorrhizal,Corticioid,NaN,Highly Probable,NaN,"b'Rinaldi AC, et al. 2008. Fungal Diversity 33..."
a618c5f336416bda99f2f5d289dd74c6,k__Fungi; p__Ascomycota; c__Leotiomycetes; o__...,Pseudeurotiaceae,9,Saprotroph,Plant Saprotroph-Wood Saprotroph,Microfungus,NaN,Probable,Saprobic on woody tissue and rotting vegetatio...,"b'Cannon PF, Kirk PM. 2007. Fungal Families of..."
ad520cb61c8760c17a08cd2d6327b35f,k__Fungi; p__Basidiomycota; c__Tremellomycetes...,-,-,-,-,-,-,-,Unassigned,-


In [5]:
df_tax['group'] = df_tax['Guild'].fillna('Other')
df_tax['group'] = df_tax['group'].apply(lambda x: x if x != "-" else "Other")
df_tax['group'] = df_tax.apply(lambda x: x['group'] if x['Confidence Ranking'] in ["Probable","Highly Probable"] else "Other", axis=1)
df_tax['group'] = df_tax['group'].apply(lambda x: 'Mixed' if '-' in str(x) else str(x))
df_tax.head()

df_guilds = df_tax[['group']]
df_guilds

,group
#OTU ID,
77b72457c1ffe5587d944f4faf895c1f,Ectomycorrhizal
1b0809805096c16c7a7126f5ae3aef8a,Mixed
7b5e91b75899fc74da9b7d623f2b9c6e,Ectomycorrhizal
a618c5f336416bda99f2f5d289dd74c6,Mixed
ad520cb61c8760c17a08cd2d6327b35f,Other
...,...
e250a81c33b866cde2a893e42dacfa15,Other
49fb52ae8ed9a41175b4c680f1baa0e6,Other
fd9bebc7de0e43c6732417e2a4101ea1,Other


## Subset soil samples

In [6]:
columns_soil = [ele for ele in df_abu if 'Soil' in ele]
df_abu_soil = df_abu[columns_soil]
print(df_abu_soil.shape)
df_abu_soil.head()

(7320, 42)


,003-ESSI-001-S-Soil-ITS,006-ESSI-002-S-Soil-ITS,009-ESSI-003-S-Soil-ITS,012-ESSI-004-S-Soil-ITS,015-ESSI-005-S-Soil-ITS,018-AMOS-001-S-Soil-ITS,021-AMOS-002-S-Soil-ITS,024-AMOS-003-S-Soil-ITS,027-AMOS-004-S-Soil-ITS,030-AMOS-005-S-Soil-ITS,...,102-FLOR1-3-3-Soil-ITS,105-FLOR1-4-3-Soil-ITS,108-FLOR1-5-3-Soil-ITS,114-Santiago-2-3-Soil-ITS,117-Santiago-3-3-Soil-ITS,120-Santiago-4-3-Soil-ITS,123-Santiago-5-3-Soil-ITS,129-Santiago-7-3-Soil-ITS,135-Santiago-9-3-Soil-ITS,138-Santiago-10-3-Soil-ITS
#OTU ID,,,,,,,,,,,,,,,,,,,,,
77b72457c1ffe5587d944f4faf895c1f,941.0,3898.0,1602.0,861.0,609.0,2203.0,2435.0,1370.0,1587.0,1540.0,...,1098.0,2466.0,2646.0,2204.0,6416.0,2308.0,2421.0,1668.0,1008.0,5687.0
1b0809805096c16c7a7126f5ae3aef8a,550.0,3233.0,1138.0,601.0,367.0,1041.0,1355.0,903.0,956.0,949.0,...,765.0,2300.0,1760.0,1247.0,3505.0,942.0,1231.0,833.0,559.0,3430.0
7b5e91b75899fc74da9b7d623f2b9c6e,585.0,2477.0,982.0,452.0,245.0,1225.0,1527.0,808.0,990.0,975.0,...,802.0,1596.0,1390.0,1257.0,4665.0,1093.0,1418.0,974.0,623.0,3814.0
a618c5f336416bda99f2f5d289dd74c6,0.0,41.0,164.0,51.0,54.0,86.0,47.0,45.0,41.0,49.0,...,4546.0,719.0,2473.0,267.0,0.0,2785.0,6833.0,0.0,6877.0,37.0
ad520cb61c8760c17a08cd2d6327b35f,157.0,641.0,378.0,174.0,91.0,2070.0,1182.0,531.0,893.0,596.0,...,5970.0,587.0,341.0,675.0,873.0,1152.0,2109.0,332.0,581.0,741.0


# Sum abundances across guilds

In [7]:
df_guilds.head()

,group
#OTU ID,
77b72457c1ffe5587d944f4faf895c1f,Ectomycorrhizal
1b0809805096c16c7a7126f5ae3aef8a,Mixed
7b5e91b75899fc74da9b7d623f2b9c6e,Ectomycorrhizal
a618c5f336416bda99f2f5d289dd74c6,Mixed
ad520cb61c8760c17a08cd2d6327b35f,Other


In [8]:
print(df_guilds.shape)
print(df_abu_soil.shape)
df_abu_soil.head()

(7320, 1)
(7320, 42)


,003-ESSI-001-S-Soil-ITS,006-ESSI-002-S-Soil-ITS,009-ESSI-003-S-Soil-ITS,012-ESSI-004-S-Soil-ITS,015-ESSI-005-S-Soil-ITS,018-AMOS-001-S-Soil-ITS,021-AMOS-002-S-Soil-ITS,024-AMOS-003-S-Soil-ITS,027-AMOS-004-S-Soil-ITS,030-AMOS-005-S-Soil-ITS,...,102-FLOR1-3-3-Soil-ITS,105-FLOR1-4-3-Soil-ITS,108-FLOR1-5-3-Soil-ITS,114-Santiago-2-3-Soil-ITS,117-Santiago-3-3-Soil-ITS,120-Santiago-4-3-Soil-ITS,123-Santiago-5-3-Soil-ITS,129-Santiago-7-3-Soil-ITS,135-Santiago-9-3-Soil-ITS,138-Santiago-10-3-Soil-ITS
#OTU ID,,,,,,,,,,,,,,,,,,,,,
77b72457c1ffe5587d944f4faf895c1f,941.0,3898.0,1602.0,861.0,609.0,2203.0,2435.0,1370.0,1587.0,1540.0,...,1098.0,2466.0,2646.0,2204.0,6416.0,2308.0,2421.0,1668.0,1008.0,5687.0
1b0809805096c16c7a7126f5ae3aef8a,550.0,3233.0,1138.0,601.0,367.0,1041.0,1355.0,903.0,956.0,949.0,...,765.0,2300.0,1760.0,1247.0,3505.0,942.0,1231.0,833.0,559.0,3430.0
7b5e91b75899fc74da9b7d623f2b9c6e,585.0,2477.0,982.0,452.0,245.0,1225.0,1527.0,808.0,990.0,975.0,...,802.0,1596.0,1390.0,1257.0,4665.0,1093.0,1418.0,974.0,623.0,3814.0
a618c5f336416bda99f2f5d289dd74c6,0.0,41.0,164.0,51.0,54.0,86.0,47.0,45.0,41.0,49.0,...,4546.0,719.0,2473.0,267.0,0.0,2785.0,6833.0,0.0,6877.0,37.0
ad520cb61c8760c17a08cd2d6327b35f,157.0,641.0,378.0,174.0,91.0,2070.0,1182.0,531.0,893.0,596.0,...,5970.0,587.0,341.0,675.0,873.0,1152.0,2109.0,332.0,581.0,741.0


In [22]:
df_sum = df_abu_soil.groupby(df_guilds['group']).sum()
df_sum.to_csv("guilds_01_read_stands.csv", sep=",", header=True, index=True)

df_sum.head()

,003-ESSI-001-S-Soil-ITS,006-ESSI-002-S-Soil-ITS,009-ESSI-003-S-Soil-ITS,012-ESSI-004-S-Soil-ITS,015-ESSI-005-S-Soil-ITS,018-AMOS-001-S-Soil-ITS,021-AMOS-002-S-Soil-ITS,024-AMOS-003-S-Soil-ITS,027-AMOS-004-S-Soil-ITS,030-AMOS-005-S-Soil-ITS,...,102-FLOR1-3-3-Soil-ITS,105-FLOR1-4-3-Soil-ITS,108-FLOR1-5-3-Soil-ITS,114-Santiago-2-3-Soil-ITS,117-Santiago-3-3-Soil-ITS,120-Santiago-4-3-Soil-ITS,123-Santiago-5-3-Soil-ITS,129-Santiago-7-3-Soil-ITS,135-Santiago-9-3-Soil-ITS,138-Santiago-10-3-Soil-ITS
group,,,,,,,,,,,,,,,,,,,,,
Animal Pathogen,3.0,19.0,165.0,49.0,0.0,33.0,61.0,50.0,29.0,57.0,...,4.0,87.0,9.0,40.0,0.0,157.0,27.0,167.0,43.0,45.0
Arbuscular Mycorrhizal,0.0,0.0,16.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18.0,0.0,0.0,11.0,0.0,0.0
Dung Saprotroph,0.0,0.0,10.0,0.0,0.0,0.0,21.0,0.0,81.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0
Ectomycorrhizal,11916.0,11282.0,11175.0,12691.0,7141.0,22853.0,12884.0,18160.0,6747.0,12390.0,...,13020.0,7617.0,12996.0,10321.0,17093.0,12193.0,6382.0,16488.0,6337.0,16019.0
Endophyte,160.0,177.0,388.0,149.0,104.0,70.0,79.0,0.0,45.0,43.0,...,110.0,48.0,75.0,199.0,123.0,183.0,110.0,111.0,67.0,130.0


In [23]:
dmeta.head()

,long.name,short.name,short.code,source,site,marker,Cluster,Country,Name.of.the.sample,Sample.code,no.anal,Lattitude,Longitude,source_marker,SampleID
X001.ESSI.001.S.Root.ITS,X001.ESSI.001.S.Root.ITS,ESSI.001.S,ESSI.001.S,Root,ESSI,ITS,NENA,Canada,NaN,NaN,NaN,NaN,NaN,Root_ITS,001-ESSI-001-S-Root-ITS
X002.ESSI.001.S.Rhizo.ITS,X002.ESSI.001.S.Rhizo.ITS,ESSI.001.S,ESSI.001.S,Rhizo,ESSI,ITS,NENA,Canada,NaN,NaN,NaN,NaN,NaN,Rhizo_ITS,002-ESSI-001-S-Rhizo-ITS
X003.ESSI.001.S.Soil.ITS,X003.ESSI.001.S.Soil.ITS,ESSI.001.S,ESSI.001.S,Soil,ESSI,ITS,NENA,Canada,ESSI-001 S,ESSI-001,19-130-001,48.341226,-69.399585,Soil_ITS,003-ESSI-001-S-Soil-ITS
X005.ESSI.002.S.Rhizo.ITS,X005.ESSI.002.S.Rhizo.ITS,ESSI.002.S,ESSI.002.S,Rhizo,ESSI,ITS,NENA,Canada,NaN,NaN,NaN,NaN,NaN,Rhizo_ITS,005-ESSI-002-S-Rhizo-ITS
X006.ESSI.002.S.Soil.ITS,X006.ESSI.002.S.Soil.ITS,ESSI.002.S,ESSI.002.S,Soil,ESSI,ITS,NENA,Canada,ESSI-002 S,ESSI-002,19-130-002,48.341226,-69.399585,Soil_ITS,006-ESSI-002-S-Soil-ITS


In [24]:
dt_sum = df_sum.T
stack_guilds = dt_sum.stack().reset_index().rename(columns={'level_0' : 'sample', 0 : 'count'})
df_meta_summed = pd.merge(stack_guilds, dmeta, left_on = 'sample', right_on = 'SampleID', how='left')
df_meta_summed

,sample,group,count,long.name,short.name,short.code,source,site,marker,Cluster,Country,Name.of.the.sample,Sample.code,no.anal,Lattitude,Longitude,source_marker,SampleID
0,003-ESSI-001-S-Soil-ITS,Animal Pathogen,3.0,X003.ESSI.001.S.Soil.ITS,ESSI.001.S,ESSI.001.S,Soil,ESSI,ITS,NENA,Canada,ESSI-001 S,ESSI-001,19-130-001,48.341226,-69.399585,Soil_ITS,003-ESSI-001-S-Soil-ITS
1,003-ESSI-001-S-Soil-ITS,Arbuscular Mycorrhizal,0.0,X003.ESSI.001.S.Soil.ITS,ESSI.001.S,ESSI.001.S,Soil,ESSI,ITS,NENA,Canada,ESSI-001 S,ESSI-001,19-130-001,48.341226,-69.399585,Soil_ITS,003-ESSI-001-S-Soil-ITS
2,003-ESSI-001-S-Soil-ITS,Dung Saprotroph,0.0,X003.ESSI.001.S.Soil.ITS,ESSI.001.S,ESSI.001.S,Soil,ESSI,ITS,NENA,Canada,ESSI-001 S,ESSI-001,19-130-001,48.341226,-69.399585,Soil_ITS,003-ESSI-001-S-Soil-ITS
3,003-ESSI-001-S-Soil-ITS,Ectomycorrhizal,11916.0,X003.ESSI.001.S.Soil.ITS,ESSI.001.S,ESSI.001.S,Soil,ESSI,ITS,NENA,Canada,ESSI-001 S,ESSI-001,19-130-001,48.341226,-69.399585,Soil_ITS,003-ESSI-001-S-Soil-ITS
4,003-ESSI-001-S-Soil-ITS,Endophyte,160.0,X003.ESSI.001.S.Soil.ITS,ESSI.001.S,ESSI.001.S,Soil,ESSI,ITS,NENA,Canada,ESSI-001 S,ESSI-001,19-130-001,48.341226,-69.399585,Soil_ITS,003-ESSI-001-S-Soil-ITS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,138-Santiago-10-3-Soil-ITS,Orchid Mycorrhizal,0.0,X138.Santiago.10.3.Soil.ITS,Santiago.10.3,Santiago.10,Soil,Santiago,ITS,MX,Mexico,Santiago 10-3,SAAA-10,19-129-010,25.028922,-105.713823,Soil_ITS,138-Santiago-10-3-Soil-ITS
668,138-Santiago-10-3-Soil-ITS,Other,14727.0,X138.Santiago.10.3.Soil.ITS,Santiago.10.3,Santiago.10,Soil,Santiago,ITS,MX,Mexico,Santiago 10-3,SAAA-10,19-129-010,25.028922,-105.713823,Soil_ITS,138-Santiago-10-3-Soil-ITS
669,138-Santiago-10-3-Soil-ITS,Plant Pathogen,0.0,X138.Santiago.10.3.Soil.ITS,Santiago.10.3,Santiago.10,Soil,Santiago,ITS,MX,Mexico,Santiago 10-3,SAAA-10,19-129-010,25.028922,-105.713823,Soil_ITS,138-Santiago-10-3-Soil-ITS
670,138-Santiago-10-3-Soil-ITS,Undefined Saprotroph,2713.0,X138.Santiago.10.3.Soil.ITS,Santiago.10.3,Santiago.10,Soil,Santiago,ITS,MX,Mexico,Santiago 10-3,SAAA-10,19-129-010,25.028922,-105.713823,Soil_ITS,138-Santiago-10-3-Soil-ITS


In [25]:
df_meta_summed.to_csv("guilds_01_read_stands_meta.csv", sep=",", header=True, index=False)